## **Training the model**

In [1]:
import os
import sys 
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader

from torchvision.transforms import ToTensor
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
import timm
import segmentation_models_pytorch as smp
import wandb
from tqdm import tqdm

from dataset import *

sys.path.append('/'.join(os.getcwd().split('\\')[:2]) + '/utils')
from log import *

from model import unet_model 
from train import *
from mask2rgb import * 
from mask2string import *

C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_images_path = []
for root, dirs, files in os.walk(TRAIN_IMAGES_DIR):
    for file in files:
        path = os.path.join(root,file)
        train_images_path.append(path)
        
len(train_images_path)

1000

In [4]:
train_masks_path = []
for root, dirs, files in os.walk(TRAIN_MASKS_DIR):
    for file in files:
        path = os.path.join(root,file)
        train_masks_path.append(path)
        
len(train_masks_path)

1000

In [5]:
dataset = NeoPolypDataset(
    img_dir=TRAIN_IMAGES_DIR, 
    label_dir=TRAIN_MASKS_DIR, 
    resize=(256, 256), 
    transform=None
    )

In [6]:
images_data = []
labels_data = []
for x,y in dataset:
    images_data.append(x)
    labels_data.append(y)

In [7]:
model = unet_model
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [8]:
train_size = int(TRAIN_RATIO * len(images_data))
val_size = len(images_data) - train_size
train_dataset = CustomDataset(images_data[:train_size], labels_data[:train_size], transform=train_transformation)
val_dataset = CustomDataset(images_data[train_size:], labels_data[train_size:], transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=BATCHSIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE, shuffle=True)

In [9]:
wandb.login(
    key = "25283834ecbe7bd282505b0721ea3adcd8e789d3",
)
wandb.init(
    project = "UNet for Colonoscopy Polyp Segmentation",
    name=f'unet-{ENCODER_NAME}'
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fisherman611 (fisherman611-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Administrator\_netrc


In [10]:
train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(),
    num_epochs=EPOCHS,
    device=device,
    save_path=SAVE_PATH,
    wandb=wandb  # Pass wandb object if logging is required, or set to None
)

Training Progress:   0%|          | 0/200 [00:39<?, ?it/s, Train Loss=0.59, Val Loss=0.298]

Epoch [1/200], Validation Loss: 0.2977014390


Training Progress:   0%|          | 1/200 [01:20<2:17:15, 41.38s/it, Train Loss=0.233, Val Loss=0.196]

Epoch [2/200], Validation Loss: 0.1959930463


Training Progress:   1%|          | 2/200 [02:00<2:14:18, 40.70s/it, Train Loss=0.147, Val Loss=0.161]

...

Epoch [198/200], Validation Loss: 0.0598422836


Training Progress: 100%|█████████▉| 199/200 [2:10:20<00:39, 39.20s/it, Train Loss=0.00432, Val Loss=0.0595]

Epoch [199/200], Validation Loss: 0.0595428152


Training Progress: 100%|██████████| 200/200 [2:10:59<00:00, 39.30s/it, Train Loss=0.00456, Val Loss=0.0613]

Epoch [200/200], Validation Loss: 0.0613406011


In [11]:
checkpoint = torch.load(SAVE_PATH)
model.load_state_dict(checkpoint['model'])
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13264\2789154915.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(SAVE_PATH)


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [12]:
model.eval()
for i in os.listdir(TEST_IMAGES_DIR):
    img_path = os.path.join(TEST_IMAGES_DIR, i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (256, 256))
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask2rgb(mask, COLOR_DICT)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"{PREDICTION_PATH}{i}", mask_rgb)
    

In [13]:
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']
df.to_csv(SUBMISSION_PATH, index=False)

d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\019410b1fcf0625f608b4ce97629ab55.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\02fa602bb3c7abacdbd7e6afd56ea7bc.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\0398846f67b5df7cdf3f33c3ca4d5060.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\05734fbeedd0f9da760db74a29abdb04.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\05b78a91391adc0bb223c4eaf3372eae.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\0619ebebe9e9c9d00a4262b4fe4a5a95.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\0626ab4ec3d46e602b296cc5cfd263f1.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\0a0317371a966bf4b3466463a3c64db1.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segmentation/unet_resnet152_prediction\0a5f3601ad4f13ccf1f4b331a412fc44.jpeg
d:/UNet-for-Colonoscopy-Polyp-Segment